In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, classification_report
from scipy.sparse import hstack
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.base import BaseEstimator, TransformerMixin
import warnings
warnings.filterwarnings('ignore')

In [2]:
class FeatureCombiner(BaseEstimator, TransformerMixin):
    """Combine TF-IDF text features with categorical features"""
    
    def __init__(self, max_features=1000, min_df=2, max_df=0.8, 
                 ngram_range=(1, 1), remove_crash=False):
        self.max_features = max_features
        self.min_df = min_df
        self.max_df = max_df
        self.ngram_range = ngram_range
        self.remove_crash = remove_crash
        self.tfidf = None
        self.encoder_component = None
        self.encoder_product = None
        
    def fit(self, X, y=None):
        # Build stopwords list
        if self.remove_crash:
            from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
            # Add crash and its variants to stopwords
            crash_variants = ['crash', 'crashes', 'crashed', 'crashing', 'crasher']
            stop_words = list(ENGLISH_STOP_WORDS) + crash_variants
            print(f"  ⚠️  Removing from vocabulary: {crash_variants}")
        else:
            stop_words = 'english'
        
        # TF-IDF for text
        self.tfidf = TfidfVectorizer(
            max_features=self.max_features,
            min_df=self.min_df,
            max_df=self.max_df,
            ngram_range=self.ngram_range,
            stop_words=stop_words
        )
        self.tfidf.fit(X['text_processed'])
        
        # OrdinalEncoder for categorical
        self.encoder_component = OrdinalEncoder(
            handle_unknown='use_encoded_value',
            unknown_value=-1
        )
        self.encoder_component.fit(X[['component_name']])
        
        self.encoder_product = OrdinalEncoder(
            handle_unknown='use_encoded_value',
            unknown_value=-1
        )
        self.encoder_product.fit(X[['product_name']])
        
        return self
    
    def transform(self, X):
        X_tfidf = self.tfidf.transform(X['text_processed'])
        component_encoded = self.encoder_component.transform(X[['component_name']])
        product_encoded = self.encoder_product.transform(X[['product_name']])
        text_length = X['text_length'].values.reshape(-1, 1)
        X_combined = hstack([X_tfidf, component_encoded, product_encoded, text_length])
        return X_combined

In [3]:
data = pd.read_csv('../data/bugs_preprocessed.csv')
data = data[
    data['text_processed'].notna() & 
    (data['text_processed'].str.strip() != '')
].copy().reset_index(drop=True)
data['text_length'] = data['text_processed'].str.split().str.len()

X_combined_data = pd.DataFrame({
    'text_processed': data['text_processed'],
    'component_name': data['component_name'],
    'product_name': data['product_name'],
    'text_length': data['text_length']
})
y_original = data['severity_category']

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_original)

# Split
X_dev, X_test, y_dev, y_test = train_test_split(
    X_combined_data, y_encoded, 
    test_size=0.2, 
    random_state=42, 
    stratify=y_encoded
)

print(f"\nDataset: {len(data)} samples")
print(f"Dev: {len(X_dev)}, Test: {len(X_test)}")


Dataset: 9998 samples
Dev: 7998, Test: 2000


In [4]:
print("\n" + "="*70)
print("EXPERIMENT 1: BASELINE (WITH 'crash')")
print("="*70)

baseline_pipeline = ImbPipeline([
    ('feature_combiner', FeatureCombiner(
        max_features=1000,
        ngram_range=(1, 2),
        remove_crash=False  # KEEP crash
    )),
    ('smote', SMOTE(random_state=42, k_neighbors=3)),
    ('classifier', RandomForestClassifier(
        n_estimators=200,
        max_depth=20,
        random_state=42,
        n_jobs=-1
    ))
])

print("Training baseline...")
baseline_pipeline.fit(X_dev, y_dev)

y_pred_baseline = baseline_pipeline.predict(X_test)
y_pred_baseline_labels = label_encoder.inverse_transform(y_pred_baseline)
y_test_labels = label_encoder.inverse_transform(y_test)

baseline_f1_macro = f1_score(y_test_labels, y_pred_baseline_labels, average='macro')
baseline_f1_weighted = f1_score(y_test_labels, y_pred_baseline_labels, average='weighted')
baseline_f1_per_class = f1_score(y_test_labels, y_pred_baseline_labels, 
                                  average=None, labels=sorted(label_encoder.classes_))

print(f"\n✅ Baseline Results:")
print(f"   F1-Macro:    {baseline_f1_macro:.4f}")
print(f"   F1-Weighted: {baseline_f1_weighted:.4f}")
print("\n   Per-class F1:")
for idx, cls in enumerate(sorted(label_encoder.classes_)):
    print(f"     {cls:12s}: {baseline_f1_per_class[idx]:.4f}")



EXPERIMENT 1: BASELINE (WITH 'crash')
Training baseline...

✅ Baseline Results:
   F1-Macro:    0.3010
   F1-Weighted: 0.6768

   Per-class F1:
     blocker     : 0.0769
     critical    : 0.4856
     major       : 0.1977
     minor       : 0.0952
     normal      : 0.8179
     trivial     : 0.1325


In [5]:
print("\n" + "="*70)
print("EXPERIMENT 2: ABLATION (WITHOUT 'crash')")
print("="*70)

ablation_pipeline = ImbPipeline([
    ('feature_combiner', FeatureCombiner(
        max_features=1000,
        ngram_range=(1, 2),
        remove_crash=True  # REMOVE crash
    )),
    ('smote', SMOTE(random_state=42, k_neighbors=3)),
    ('classifier', RandomForestClassifier(
        n_estimators=200,
        max_depth=20,
        random_state=42,
        n_jobs=-1
    ))
])

print("Training ablation model...")
ablation_pipeline.fit(X_dev, y_dev)

y_pred_ablation = ablation_pipeline.predict(X_test)
y_pred_ablation_labels = label_encoder.inverse_transform(y_pred_ablation)

ablation_f1_macro = f1_score(y_test_labels, y_pred_ablation_labels, average='macro')
ablation_f1_weighted = f1_score(y_test_labels, y_pred_ablation_labels, average='weighted')
ablation_f1_per_class = f1_score(y_test_labels, y_pred_ablation_labels, 
                                  average=None, labels=sorted(label_encoder.classes_))

print(f"\n✅ Ablation Results:")
print(f"   F1-Macro:    {ablation_f1_macro:.4f}")
print(f"   F1-Weighted: {ablation_f1_weighted:.4f}")
print("\n   Per-class F1:")
for idx, cls in enumerate(sorted(label_encoder.classes_)):
    print(f"     {cls:12s}: {ablation_f1_per_class[idx]:.4f}")



EXPERIMENT 2: ABLATION (WITHOUT 'crash')
Training ablation model...
  ⚠️  Removing from vocabulary: ['crash', 'crashes', 'crashed', 'crashing', 'crasher']

✅ Ablation Results:
   F1-Macro:    0.2748
   F1-Weighted: 0.6597

   Per-class F1:
     blocker     : 0.0741
     critical    : 0.3276
     major       : 0.1697
     minor       : 0.1163
     normal      : 0.8085
     trivial     : 0.1529


In [6]:
print("\n" + "="*70)
print("📊 COMPARISON: IMPACT OF REMOVING 'CRASH'")
print("="*70)

f1_macro_diff = baseline_f1_macro - ablation_f1_macro
f1_weighted_diff = baseline_f1_weighted - ablation_f1_weighted
f1_macro_pct = (f1_macro_diff / baseline_f1_macro) * 100 if baseline_f1_macro > 0 else 0
f1_weighted_pct = (f1_weighted_diff / baseline_f1_weighted) * 100 if baseline_f1_weighted > 0 else 0

print("\nOverall Metrics:")
print(f"{'Metric':<20} {'Baseline':<12} {'Ablation':<12} {'Δ Absolute':<15} {'Δ Percent':<12}")
print("-"*75)
print(f"{'F1-Macro':<20} {baseline_f1_macro:<12.4f} {ablation_f1_macro:<12.4f} "
      f"{f1_macro_diff:+<15.4f} {f1_macro_pct:+<12.2f}%")
print(f"{'F1-Weighted':<20} {baseline_f1_weighted:<12.4f} {ablation_f1_weighted:<12.4f} "
      f"{f1_weighted_diff:+<15.4f} {f1_weighted_pct:+<12.2f}%")

print("\n\nPer-Class Impact:")
print(f"{'Class':<15} {'Baseline':<12} {'Ablation':<12} {'Δ Absolute':<15} {'Δ Percent':<12}")
print("-"*75)

for idx, cls in enumerate(sorted(label_encoder.classes_)):
    base_f1 = baseline_f1_per_class[idx]
    abl_f1 = ablation_f1_per_class[idx]
    diff = base_f1 - abl_f1
    pct = (diff / base_f1 * 100) if base_f1 > 0 else 0
    print(f"{cls:<15} {base_f1:<12.4f} {abl_f1:<12.4f} {diff:+<15.4f} {pct:+<12.2f}%")


📊 COMPARISON: IMPACT OF REMOVING 'CRASH'

Overall Metrics:
Metric               Baseline     Ablation     Δ Absolute      Δ Percent   
---------------------------------------------------------------------------
F1-Macro             0.3010       0.2748       0.0261+++++++++ 8.68++++++++%
F1-Weighted          0.6768       0.6597       0.0171+++++++++ 2.52++++++++%


Per-Class Impact:
Class           Baseline     Ablation     Δ Absolute      Δ Percent   
---------------------------------------------------------------------------
blocker         0.0769       0.0741       0.0028+++++++++ 3.70++++++++%
critical        0.4856       0.3276       0.1580+++++++++ 32.54+++++++%
major           0.1977       0.1697       0.0280+++++++++ 14.15+++++++%
minor           0.0952       0.1163       -0.0210++++++++ -22.09++++++%
normal          0.8179       0.8085       0.0094+++++++++ 1.15++++++++%
trivial         0.1325       0.1529       -0.0204++++++++ -15.41++++++%


In [7]:
print("\n" + "="*70)
print("📋 SUMMARY")
print("="*70)

if f1_macro_diff > 0.01:  # Meaningful drop
    print(f"\n✅ HYPOTHESIS CONFIRMED:")
    print(f"   Removing 'crash' caused {abs(f1_macro_pct):.1f}% performance drop in F1-Macro")
    print(f"   This validates that 'crash' is a CRITICAL feature for severity detection")
    
    # Find most impacted class
    per_class_diffs = baseline_f1_per_class - ablation_f1_per_class
    most_impacted_idx = np.argmax(per_class_diffs)
    most_impacted_class = sorted(label_encoder.classes_)[most_impacted_idx]
    most_impacted_drop = per_class_diffs[most_impacted_idx]
    
    print(f"\n   Most impacted class: {most_impacted_class}")
    print(f"   F1-score drop: {most_impacted_drop:.4f}")
    
elif f1_macro_diff < -0.01:  # Improved without crash
    print(f"\n⚠️  UNEXPECTED RESULT:")
    print(f"   Model performed BETTER without 'crash' ({abs(f1_macro_pct):.1f}% improvement)")
    print(f"   This suggests 'crash' may be causing overfitting or the model is")
    print(f"   learning from other correlated features")
    
else:  # No meaningful change
    print(f"\n❓ INCONCLUSIVE:")
    print(f"   Removing 'crash' had minimal impact ({abs(f1_macro_pct):.1f}% change)")
    print(f"   Model may be relying on correlated features like 'hang', 'freeze', etc.")

print("\n" + "="*70)
print("✅ ABLATION STUDY COMPLETE!")
print("="*70)

# Save quick summary
summary = f"""
ABLATION STUDY SUMMARY
======================

Hypothesis: Removing 'crash' (most important feature) should degrade performance

Results:
- Baseline F1-Macro:    {baseline_f1_macro:.4f}
- Ablation F1-Macro:    {ablation_f1_macro:.4f}
- Difference:           {f1_macro_diff:+.4f} ({f1_macro_pct:+.2f}%)

- Baseline F1-Weighted: {baseline_f1_weighted:.4f}
- Ablation F1-Weighted: {ablation_f1_weighted:.4f}
- Difference:           {f1_weighted_diff:+.4f} ({f1_weighted_pct:+.2f}%)

Conclusion: {'CONFIRMED - crash is critical' if f1_macro_diff > 0.01 else 'INCONCLUSIVE or UNEXPECTED'}
"""

print("\n" + summary)

with open('../results/ablation_study_quick_summary.txt', 'w') as f:
    f.write(summary)

print("📄 Summary saved: ../results/ablation_study_quick_summary.txt")


📋 SUMMARY

✅ HYPOTHESIS CONFIRMED:
   Removing 'crash' caused 8.7% performance drop in F1-Macro
   This validates that 'crash' is a CRITICAL feature for severity detection

   Most impacted class: critical
   F1-score drop: 0.1580

✅ ABLATION STUDY COMPLETE!


ABLATION STUDY SUMMARY

Hypothesis: Removing 'crash' (most important feature) should degrade performance

Results:
- Baseline F1-Macro:    0.3010
- Ablation F1-Macro:    0.2748
- Difference:           +0.0261 (+8.68%)

- Baseline F1-Weighted: 0.6768
- Ablation F1-Weighted: 0.6597
- Difference:           +0.0171 (+2.52%)

Conclusion: CONFIRMED - crash is critical

📄 Summary saved: ../results/ablation_study_quick_summary.txt
